<a href="https://colab.research.google.com/github/ThisWillFall/Proyecto-final-IA/blob/main/Clasificaci%C3%B3n_de_g%C3%A9neros_musicales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Limpieza de datos

In [1]:
import pandas as pd
import numpy as np

Data=pd.read_csv("train.csv",header=0,sep=",")
Data=Data.drop(['Artist Name','Track Name','Popularity','key','mode','tempo','duration_in min/ms','time_signature','loudness','liveness'],axis=1) 
Data=Data.dropna(axis=0,how='any')

Y=Data['Class'].values
X=Data[['danceability','energy','speechiness','acousticness','instrumentalness','valence']].values

## 2. División del conjunto y normalización

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=0)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


## 3. PCA

In [36]:
from sklearn import decomposition
from sklearn.decomposition import PCA, KernelPCA
for n in [1,2,3,4,5]:
  pca = decomposition.PCA(n_components=n,whiten=True,svd_solver='arpack')
  pca.fit(X_train)
  Xt = pca.transform(X_train)
  print("Dimensiones:",n)
  print("Información:",sum(pca.explained_variance_ratio_))

Dimensiones: 1
Información: 0.33600601342264
Dimensiones: 2
Información: 0.5813924285053956
Dimensiones: 3
Información: 0.7441033697319595
Dimensiones: 4
Información: 0.8798034416466224
Dimensiones: 5
Información: 0.9653891673991161


## 3. Grid Search para implementar clasificador de Random Forest

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

param= {'n_estimators':[10,100,1000,10000],'criterion':['gini','entropy']}
grid = GridSearchCV(RandomForestClassifier(), param,cv=KFold(n_splits=2))
grid.fit(X_train, y_train)

print('Los mejores parametros fueron:',grid.best_params_)
print(grid.best_estimator_)

Los mejores parametros fueron: {'criterion': 'gini', 'n_estimators': 10000}
RandomForestClassifier(n_estimators=10000)


## 4. Reentrenamiento con el mejor método de hiperparametros

In [4]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize

rfc = RandomForestClassifier(criterion='gini',n_estimators=10000)
rfc.fit(X_train, y_train)
y_test_predicted = rfc.predict(X_test)
MCC = matthews_corrcoef(y_test, y_test_predicted)
F1=f1_score(y_test, y_test_predicted, average='macro')
AUC=roc_auc_score(label_binarize(y_test,classes=[0,1,2,3,4,5,6,7,8,9,10]),label_binarize(y_test_predicted,classes=[0,1,2,3,4,5,6,7,8,9,10]),multi_class='ovr')
print("El MCC es de ",MCC)
print("El AUC-ROC es de ",AUC)
print("El F1 es de ",F1)

El MCC es de  0.2632206695446134
El AUC-ROC es de  0.6404856601051535
El F1 es de  0.3522986276632452


## 5. Grid Search para implementar regresión logistica

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

param= {'C':[1/1000,1/100,1/10,1],'l1_ratio':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
grid = GridSearchCV(LogisticRegression(solver='saga', penalty='elasticnet',max_iter=1000), param,cv=KFold(n_splits=2))
grid.fit(X_train, y_train)

print('Los mejores parametros fueron:',grid.best_params_)
print(grid.best_estimator_)

Los mejores parametros fueron: {'C': 0.1, 'l1_ratio': 0.6}
LogisticRegression(C=0.1, l1_ratio=0.6, max_iter=1000, penalty='elasticnet',
                   solver='saga')


## 6. Reentrenamiento con el mejor método de hiperparametros

In [14]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix

lr = LogisticRegression(C=0.1, l1_ratio=0.6, max_iter=1000, penalty='elasticnet',
                   solver='saga')
lr.fit(X_train, y_train)
y_test_predicted = lr.predict(X_test)
MCC = matthews_corrcoef(y_test, y_test_predicted)
F1=f1_score(y_test, y_test_predicted, average='macro');
AUC=roc_auc_score(label_binarize(y_test,classes=[0,1,2,3,4,5,6,7,8,9,10]),label_binarize(y_test_predicted,classes=[0,1,2,3,4,5,6,7,8,9,10]),multi_class='ovr')
print("El MCC es de ",MCC)
print("El AUC-ROC es de ",AUC)
print("El F1 es de ",F1)


El MCC es de  0.28540191757750466
El AUC-ROC es de  0.6415128303934824
El F1 es de  0.34760484805804776
